# 分步微笑模拟

该Notebook将演示如何根据初始微笑像与矫治分步方案生成分步微笑照

In [ ]:
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np
import DracoPy
from PIL import Image
from io import BytesIO
import cv2
import glob
import matplotlib.pyplot as plt

## 定义调用规则

In [ ]:
# 朝厚服务请求地址，随api文档发送
base_url = "<服务请求地址>"

# 朝厚文件服务地址，随api文档发送
file_server_url = "<服务文件服务器地址>"

# 必须传入鉴权 Header。请保护好TOKEN!!! 如果泄露请立即联系我们重置，所有使用该TOKEN的任务都会向您的账户计费
zh_token = "<贵司服务Token, 随合同发送>" # 调用所有的API都必须传入token用作鉴权

user_group = "APIClient" # 用户组，一般为 APIClient

# 贵司user_id, 随api文档发送
user_id = "<贵司user_id>"

# 如果您收到了creds.json, 下面将直接读取
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # 必须指定 postfix, 即文件后缀名
                        headers={"X-ZH-TOKEN": zh_token}) # 获取带签名的上传地址
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # 返回为一个单字符串JSON "string", 这里也可以用json.loads(resp.text)

    resp = requests.put(upload_url, data) # 上传至云储存服务不需要带鉴权头

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

def run_job_and_get_results(json_call, timeout_sec):
    headers = {
      "Content-Type": "application/json",
      "X-ZH-TOKEN": zh_token
    }

    url = base_url + '/run'

    response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
    response.raise_for_status()
    create_result = response.json()
    run_id = create_result['run_id']
    print("workflow id is", run_id)
    url = base_url + f"/run/{run_id}"

    start_time = time.time()
    while time.time()-start_time < timeout_sec:
        time.sleep(0.3)
        response = requests.request("GET", url, headers=headers)
        result = response.json()
        if result['completed'] or result['failed']:
            break

    if not result['completed']:
        if result['failed']:
            raise ValueError("API failed due to " + str(result['reason_public']))
        raise TimeoutError("API timeout")

    print("API finished in {}s".format(time.time()-start_time))
    url = base_url + f"/data/{run_id}"
    response = requests.request("GET", url, headers=headers)
    return response.json()

def retrieve_mesh(mesh_file_json):
    resp = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": mesh_file_json['data']}),
                        headers={"X-ZH-TOKEN": zh_token})
    return trimesh.load(trimesh.util.wrap_as_stream(resp.content), file_type=mesh_file_json['type'])

def show_img(urn):
    imbytes = requests.get(file_server_url + f"/file/download?" + urllib.parse.urlencode({
                        "urn": urn}),
                        headers={"X-ZH-TOKEN": zh_token}).content
    imarray = np.asarray(bytearray(imbytes), dtype=np.uint8)
    plt.imshow(cv2.imdecode(imarray, cv2.IMREAD_COLOR)[...,::-1])

## 生成自动加工单与目标位

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "oral-arrangement-medical",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "upper_mesh": {"type":"drc", "data": upload_file("upper_jaw_scan.drc")},
      "lower_mesh": {"type":"ply", "data": upload_file("lower_jaw_scan.ply")},
      "ceph": upload_file("ceph.jpg"),
      "smile_photo": upload_file("face_smile.jpg"),
  },
  'output_config': {
    "u_teeth_comp": {"type": "ply"},
    "l_teeth_comp": {"type": "ply"}
  }
}
result_arrangement = run_job_and_get_results(json_call, 1200)

## 生成分步信息

In [ ]:
json_call = {
  "spec_group": "mesh-processing", # 调用的工作流组， 随API文档发送
  "spec_name": "auto-step", # 调用的工作流名称， 随API文档发送, 
  "spec_version": "1.0-snapshot", # 调用的工作流版本，随API文档发送
  "user_group": user_group,
  "user_id": user_id
}
json_call["input_data"] = {
    "upper_teeth_dict": result_arrangement["u_teeth_comp"],
    "upper_align_matrix": result_arrangement["u_align_matrix"],
    "upper_axis_matrix_dict": result_arrangement["u_axis"],
    "lower_teeth_dict": result_arrangement["l_teeth_comp"],
    "lower_align_matrix": result_arrangement["l_align_matrix"],
    "lower_axis_matrix_dict": result_arrangement["l_axis"],
    "transformation_dict": result_arrangement['transformation_dict']
}
result_step = run_job_and_get_results(json_call, 2200)

In [ ]:
original_comp = {**{k: retrieve_mesh(v) for k, v in result_arrangement["u_teeth_comp"].items()},
                 **{k: retrieve_mesh(v) for k, v in result_arrangement["l_teeth_comp"].items()}}
def show_step(step_index):
    result_mesh = None
    for k, m in original_comp.items():
        if k in result_step['result']['step_dict'][step_index]:
            result_mesh += m.copy().apply_transform(result_step['result']['step_dict'][step_index][k])
    return result_mesh

# 生成各步对应的微笑模拟

### 组装 step 字典

请注意，必须有代表初始位置的`step_0`用于与微笑像配准

In [ ]:
step_0 = {}
step_dict = {}

for k,v in result_arrangement["transformation_dict"].items():
    step_0[k] = np.eye(4,4).tolist()
step_dict['step_0'] = step_0

for i, step_transformation in enumerate(result_step["result"]['step_dict']):
    step_dict[f'step_{i+1}'] = step_transformation

### 调用算法生成微笑像

In [ ]:
json_call = {
  "spec_group": "smile",
  "spec_name": "smile-sim-to-business",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id,
  "input_data": {
      "image": upload_file("face_smile.jpg"),
      "meshes": {**result_arrangement["u_teeth_comp"], **result_arrangement["l_teeth_comp"]},
      "step": step_dict
  }
}
result_sim = run_job_and_get_results(json_call, 300)

# 可视化结果

In [ ]:
for idx, urn in result_sim['result']['image'].items():
    plt.figure()
    plt.title(idx)
    show_img(result_sim['result']['image'][idx])

In [ ]:
show_step(10)